# Trabajo Práctico: Agente para automatizar la búsqueda de repuestos

- **Curso:** DUIA - 2025, Módulo 6
- **Integrantes:** David Burckhardt, Martin Vazquez Arispe, Martin Caballero.
- **Objetivo:** Implementar un sistema inteligente que automatice la búsqueda, ranking y pedido de repuestos para una empresa distribuidora.

---
##  Índice del Notebook

1. **Consigna del trabajo**
2. **Configuracion de API Keys**

---

## 1. Consigna: agente(s) para automatizar la búsqueda de repuestos
- Dada una solicitud de repuestos específicos para una empresa distribuidora, un
agente debe identificar las especificaciones de dichos repuestos (según un
catálogo), a fin de poder buscarlos.
- El agente busca en primer lugar en el inventario de la empresa, y en en caso de
no encontrarlos (puede ser que encuentre solo algunos de ellos), debe consultar
en catálogos de proveedores.
- El sistema extrae información de las opciones encontradas, y genera un ranking
de alternativas, priorizando: 
    - Repuestos internos (si están disponibles).
    - Proveedores externos según criterios de optimización (por ej. costo-beneficio).
- Para repuestos internos: 
    - Se genera una orden de retiro del inventario y se notifica al almacén para su preparación. 
- Para repuestos externos: 
    - se envía un email automatizado al proveedor seleccionado para formalizar el pedido.

- Finalmente, se agenda la fecha estimada de entrega y detalles del pedido en el
sistema de seguimiento.
- Pueden incluirse pasos de "human in the loop" para verificar resultados antes de
tomar acciones

## 2. Configuración de API Keys y variables de entorno

Cargamos la `GROQ_API_KEY` desde el archivo `.env` e inicializamos el cliente LLM.

**Nota:** Asegúrate de tener un archivo `.env` en el directorio raíz con:
```
GROQ_API_KEY=tu_clave_aqui
```

In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

# Cargar variables de entorno
load_dotenv()

# Verificar que la API key está configurada
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    raise ValueError("GROQ_API_KEY no encontrada en .env")

# Inicializar el LLM de Groq
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.1,
    api_key=api_key
)

print("LLM de Groq inicializado correctamente")
print(f"   Modelo: {llm.model_name}")
print(f"   Temperature: {llm.temperature}")


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import json

# Create a simple prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "Mantene una conversacion con el usuario"),
    ("user", "{input}")
])

# Create the chain that guarantees JSON output
chain = prompt | llm


In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated
from langgraph.checkpoint.memory import MemorySaver


#Definimos el esquema
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

#Defino funcion del nodo
def nodo_llm(state: AgentState) -> AgentState:
    messages = state["messages"]
    response = chain.invoke(messages)
    return {
        "messages": [response],
    }

def tiene_info_suficiente(state: AgentState) -> str:
    # Logica para decidir si se continua o no
    # Por ahora, siempre va a END
    return "continue"

memory = MemorySaver()

#Defino el grafo
graph_builder = StateGraph(AgentState)

#Agrego el nodo al grafo
graph_builder.add_node("llm_node", nodo_llm)

#Conecto los nodos (Solo uno por ahora)
graph_builder.add_edge(START, "llm_node")
graph_builder.add_conditional_edges(
    "llm_node",
    tiene_info_suficiente,
    {
        "continue": "llm_node",  # Vuelve a ejecutar llm_node
        "end": END
    }
)

graph = graph_builder.compile(
    checkpointer=memory, 
    interrupt_after=["llm_node"]
)

In [ ]:
#Defino la funcion del agente
def iniciar_agente(mensaje_usuario: str):
    config = {"configurable": {"thread_id": "1"}}
    estado_inicial = AgentState(
        messages=[HumanMessage(content=mensaje_usuario)],
    )
    result = graph.invoke(estado_inicial, config)
    end = False
    while not end:
        print(result["messages"][-1].content)
        nuevo_mensaje = input("👤 Tú: ")
        if nuevo_mensaje == "salir":
            end = True
        else:
            nuevo_estado = {"messages": [HumanMessage(content=nuevo_mensaje)]}
            result = graph.invoke(nuevo_estado, config)
    return result

In [ ]:
#main
mensaje_usuario = input("👤 Tú: ")
resultado = iniciar_agente(mensaje_usuario)